In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [3]:

options = Options()
options.add_argument("--headless")
service = Service(executable_path="C:/chromedriver/chromedriver-win64/chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

driver.get("https://fbref.com/en/comps/Big5/possession/players/Big-5-European-Leagues-Stats")
time.sleep(5)  # Wait for the JS table to load

soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

table = soup.find("table", {"id": "stats_possession"}) #id extracted using inspect element
df = pd.read_html(str(table))[0]

print(df.shape)  # Expect around 2854 rows


C:\Users\shrey\AppData\Local\Temp\ipykernel_27508\1693037785.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


(2968, 32)


In [4]:

df = pd.read_html(str(table), header=[0,1])[0]

# Flatten MultiIndex columns
df.columns = [' '.join(col).strip() for col in df.columns.values]

# Remove repeated header rows if they exist
if 'Rk' in df.columns:
    df = df[df['Rk'] != 'Rk']

df.reset_index(drop=True, inplace=True)


C:\Users\shrey\AppData\Local\Temp\ipykernel_27508\2120155535.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), header=[0,1])[0]


In [5]:
def clean_prefix(text):
    if isinstance(text, str):
        m = re.match(r'^([a-z]{2,3})\s([A-Z]{3})(.*)$', text)
        if m:
            return m.group(2)
        return re.sub(r'^[a-z]{2,3}\s+', '', text)
    return text

columns_to_clean = ['Unnamed: 2_level_0 Nation', 'Unnamed: 5_level_0 Comp']

for col in columns_to_clean:
    df[col] = df[col].apply(clean_prefix)

# Step 6: Drop 'Rk' and 'Matches' columns
df.drop(columns=[
    'Unnamed: 0_level_0 Rk',
    'Unnamed: 31_level_0 Matches'
], inplace=True)

# Step 7: Rename columns to readable format also used as the header
df.columns = [
    "PLAYER", "NATION", "POS", "SQUAD", "COMP", "AGE", "BORN", "NINETIES_PLAYED",
    "TOUCHES", "TOUCHES_DEF_PEN", "TOUCHES_DEF_3RD", "TOUCHES_MID_3RD", "TOUCHES_ATT_3RD", "TOUCHES_ATT_PEN", "LIVE_TOUCHES",
    "TAKE_ONS_ATT", "TAKE_ONS_SUCC", "TAKE_ONS_SUCC_PCT", "TAKE_ONS_TKLD", "TAKE_ONS_TKLD_PCT",
    "CARRIES", "TOT_DIST", "PRG_DIST", "PRG_CARRIES", "CARRIES_FINAL_THIRD", "CARRIES_PEN_AREA", "MISCONTROL", "DISSPOSSESSED",
    "PASSES_RECEIVED", "PRG_PASSES_RECEIVED"
]

# Step 8: Save to cleaned CSV
df.to_csv("fbref_possession_2024_25_cleaned.csv", index=False)

print("Final cleaned data saved as 'fbref_possession_2024_25_cleaned.csv'")

Final cleaned data saved as 'fbref_possession_2024_25_cleaned.csv'
